In [1]:
import os
os.chdir(r'C:\Users\lipku\Documents\EE3801\Part2')

# 1. Scenario: Streaming audio

The company would like to build an in-house automatic speech transcribing tool. We record the streams of audio from one computer and another computer can receive this audio and transcribe in real-time. The system stream in audio, transcribe it using Open AI's whisper model and transcribed text is saved for reporting in real-time.

# 1.1 Single stream audio data auto-transcription

In the previous lab exercise, you have observed missing words lost in recording and transcription. In this lab, you will attempt to capture all audio streams and transcribe to text. Take note of the time taken to read, write and transcribe the audio.

You will need two notebook to run concurrently.

Producer - codes below
Consumer - stream_data_pipeline_2_consumer.ipynb

In [2]:
# # install python packages
# !pip install --upgrade pip

# # for mac users
# !brew install portaudio 
# !pip3 install pyaudio

# # for windows users
# # in wsl
# !sudo add-apt-repository ppa:therealkenc/wsl-pulseaudio sudo apt update
# !sudo apt install pulseaudio
# !pip3 install pyaudio

# # for GNU/Linux users
# !sudo apt install python3-pyaudio

In [3]:
# Check if pyaudio is installed with Conda
!conda list pyaudio


# packages in environment at C:\Users\lipku\miniconda3\envs\venv_ee3801:
#
# Name                    Version                   Build  Channel
pyaudio                   0.2.11          py311h2bbff1b_2  


# 1.2 Stream in audio

In [4]:
# Testing audio setup in this device
import pyaudio

audio = pyaudio.PyAudio()
print("audio.get_device_count():",audio.get_device_count())
for i in range(audio.get_device_count()):
    print(audio.get_device_info_by_index(i))

audio.get_device_count(): 26
{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Intel® Smart ', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'ho

In [5]:
# This is to determine which input audio and output audio you will use.
# Explore and find the right index to use for input and output in your device.
# need a device with max input channels > 0
print("Choosing my input audio:",audio.get_device_info_by_index(1))
print("Choosing my output audio:",audio.get_device_info_by_index(2))
audio.terminate()

Choosing my input audio: {'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Intel® Smart ', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
Choosing my output audio: {'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}


# 2. Load whisper model once

In [6]:
!pip3 install -U jupyter
!pip3 install -U ipywidgets
!pip3 uninstall whisper -y
!pip3 install openai-whisper
!pip3 install -U numpy
!pip3 install -U openai-whisper

  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.
torchmetrics 1.5.1 requires numpy<2.0,>1.20.0, but you have numpy 2.1.3 which is incompatible.


  Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchmetrics 1.5.1 requires numpy<2.0,>1.20.0, but you have numpy 2.0.2 which is incompatible.


In [1]:
# python 3.11.5
!pip3 install -U torch
!pip3 uninstall numpy -y
!pip3 install numpy==1.26.4
# restart kernel

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)


# 3. Read from the script as you are recording

Producers are fairly straightforward – they send messages to a topic and partition, maybe request an acknowledgment, retry if a message fails – or not – and continue. Consumers, however, can be a little more complicated.

Consumers read messages from a topic. Consumers run in a poll loop that runs indefinitely waiting for messages. Consumers can read from the beginning – they will start at the first message in the topic and read the entire history. Once caught up, the consumer will wait for new messages.

# 4. Capture every sentence in a paragraph

Using the device's audio capture the audio, record the sentence, send the audio data using Producer to a topic.

In [6]:
# if timeout add 127.0.0.1 host.docker.internal to /etc/hosts

# 4.1 Initialise Producer

In [7]:
!pip install six
!pip install kafka-python-ng

In [8]:
import six
import sys
if sys.version_info >= (3, 12, 0):
    sys.modules['kafka.vendor.six.moves'] = six.moves

In [9]:
# kafka-python Producer
from kafka import KafkaProducer
from kafka.errors import KafkaError

# produce asynchronously with callbacks
producer = KafkaProducer(bootstrap_servers=['localhost:29092','localhost:39092','localhost:49092']) 

def on_send_success(record_metadata):
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    print('I am an errback', exc_info=excp)
    # handle exception

# 4.2 Single thread audio Producer


In [17]:
# Single thread audio

import pyaudio
import wave
import numpy as np
from datetime import datetime
import whisper
import sys

FORMAT = pyaudio.paInt16
CHUNK = 1024
RECORD_SECONDS = 5
DEVICE_ID = 1

audio = pyaudio.PyAudio()
RATE = int(audio.get_device_info_by_index(DEVICE_ID)['defaultSampleRate'])
CHANNELS = int(audio.get_device_info_by_index(DEVICE_ID)['maxInputChannels'])

audio = pyaudio.PyAudio()
stream = audio.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK,
    input_device_index=DEVICE_ID
)

start_time = datetime.now()

try:
    while True:
        before_time = datetime.now()
        frames = []
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK, exception_on_overflow = False)
            frames.append(data)
        raw_data = b''.join(frames)

        # additional lines of code for functionality:
        if CHANNELS > 1:
            audio_data = np.frombuffer(raw_data, dtype=np.int16)
            audio_data = audio_data.reshape(-1, CHANNELS).mean(axis=1).astype(np.int16)
            raw_data = audio_data.tobytes()

        # produce asynchronously with callbacks
        producer.send('dataengineering', raw_data, key="audio".encode('utf-8'))\
                .add_callback(on_send_success)\
                .add_errback(on_send_error)
        print("%s audio_duration (s): %s" % (datetime.now().strftime("%d/%m/%Y, %H:%M:%S"), (datetime.now() - before_time).seconds))

        # block until all async messages are sent
        producer.flush()

        if (datetime.now() - start_time).seconds > 60: #exit program after 1min
            stream.stop_stream()
            stream.close()
            audio.terminate()
            print("* Exit program after 1min *")
            break
        
except KeyboardInterrupt as kie:
    print("* Program terminated by user *")
    stream.stop_stream()
    stream.close()
    audio.terminate()
except Exception as e:
    # print("Exception:", e)
    if stream!=None:
        stream.stop_stream()
        stream.close()
        audio.terminate()
# sys.exit(0)
# exit

04/11/2024, 13:14:02 audio_duration (s): 5
dataengineering
0
218
04/11/2024, 13:14:07 audio_duration (s): 4
dataengineering
0
219
04/11/2024, 13:14:12 audio_duration (s): 4
dataengineering
0
220
04/11/2024, 13:14:18 audio_duration (s): 4
dataengineering
0
221
04/11/2024, 13:14:23 audio_duration (s): 4
dataengineering
0
222
04/11/2024, 13:14:28 audio_duration (s): 4
dataengineering
0
223
04/11/2024, 13:14:33 audio_duration (s): 4
dataengineering
0
224
04/11/2024, 13:14:38 audio_duration (s): 4
dataengineering
0
225
04/11/2024, 13:14:43 audio_duration (s): 4
dataengineering
0
226
04/11/2024, 13:14:48 audio_duration (s): 4
dataengineering
0
227
04/11/2024, 13:14:53 audio_duration (s): 4
dataengineering
0
228
04/11/2024, 13:14:58 audio_duration (s): 4
dataengineering
0
229
04/11/2024, 13:15:03 audio_duration (s): 4
dataengineering
0
230
* Exit program after 1min *


## What did you observe from the messages sent? Submit your findings.
The messages sent from the producer are received on the consumer side in real time, with audio segments recorded and sent approximately every 5 seconds. Over a 1-minute period, 12 messages are output by the producer to the dataengineering topic before the program exits. 

However, on the consumer side, not all of the 12 messages are being received. This could be due to processing lags or an overload in transcription which led to missing data received by the consumer. This process is also susceptible to connection issues, such as network latency/kafka broker unavailability, which could possibly lead to failed message deliveries, resulting in missing data for the consumer.


# Questions to Ponder

### 1. Which principle in the good data achitecture does Kafka fulfill?
Apache Kafka fulfils several principles of good data architecture, namely: planning for failure, being an architect for scalability and being able to make reversible decisions. 
Kafka replicates data across multiple nodes, ensuring resilience in case of node failures, making it a strong choice for systems that need to handle failures gracefully.

Additionally, Kafka’s distributed architecture allows for horizontal scaling. As data volumes grow, Kafka clusters can be expanded with additional nodes, maintaining performance and accommodating higher throughput.

Lastly, Kafka retains historical data for configurable time windows, allowing for reprocessing of events and replays if needed. This is helpful for systems that need to recover from processing errors without losing data.

### 2. Can Microsoft Power App exercises do stream processing?
Microsoft Power Apps are unable to do stream processing. This is because Power Apps is primarily a low-code platform designed for building custom business applications that connect to data sources and allow users to perform actions like creating, reading, updating, and deleting records. While it can integrate with real-time data sources to provide updates in near real-time, it lacks the functionality to handle continuous, high-throughput data processing or complex event handling, which are key components of stream processing.

### 3. What is the advantages and disadvantages of using stream processing?
Stream processing provides real-time data insights, allowing for instant responses to data events. This is critical in scenarios like fraud detection, IoT monitoring, and real-time customer engagement.

By enabling real-time data insights, this also leads to improved decision making, as by analyzing data as it arrives, businesses can make better-informed decisions faster, improving operational efficiency and responsiveness. 

Additionally, stream processing frameworks like Kafka are designed to handle large volumes of data and can scale horizontally, allowing systems to grow with data needs. 

However, stream processing also has its drawbacks. Setting up and managing a stream processing pipeline can be complex as It requires careful design and often specialized knowledge to handle challenges like event ordering, latency, and fault tolerance.

Stream processing also often requires significant computational resources to handle real-time data, which can be costly. It demands more memory, storage, and processing power than traditional batch processing.

Error handling can be another issue. Since stream processing handles data in motion, recovering from errors is more challenging than with batch processing. Issues in data quality or processing logic needs to be addressed quickly, as they can propagate through the system in real time.
